# Part 1: Data Processing

### Task 1: Importing and cleaning the a sample of the FakeNewsCorpus

We will be importing the data with pandas cvs method. Cleaning the data will be done through a combination of the clean-text module and manual data cleaning, using regular expressions. 

In [ ]:
import nltk

